In [1]:
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
import pandas as pd
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE


#make a main module
if __name__ == "__main__":
        # File paths
    path_ECG = "ECG (EO, AC1, AC2).xlsx"
    path_EEG = "EEG (EO, AC1, AC2).xlsx"
    path_ratio = "Ratio of Alpha _ Beta Power.xlsx"

    # Number of people (assuming this is useful for later steps)
    num_of_people = 40

    # Read the data

    data_ECG = pd.read_excel(path_ECG, sheet_name="EO")
    data_ECG["Segment"] = "EO"

    for stress in ["AC1", "AC2"]:
        data = pd.read_excel(path_ECG, sheet_name=stress)
        data["Segment"] = stress
        data_ECG = pd.concat([data_ECG, data], ignore_index=True)

    data_EEG = pd.read_excel(path_EEG, sheet_name="Normalize")
    data_EEG = data_EEG.ffill()

    data = pd.merge(data_ECG, data_EEG, on=["Segment", "Subject (No.)"])
    for waves in ["Alpha", "Beta1", "Beta2"]:
        data_ratio = pd.read_excel(path_ratio, sheet_name=waves)
        data_ratio["Wave"] = waves

        data_ratio = data_ratio.T
        data_ratio = data_ratio.ffill()
        data_ratio = data_ratio.T

        stresses = data_ratio.iloc[0]
        new_header = data_ratio.iloc[1]

        data_ratio.columns = list(data_ratio.columns[:2]) + list(new_header[2:])
        data_ratio = data_ratio.drop([1])
        head = data_ratio.iloc[:, 0:2]


        data_ratio_ab = data_ratio.iloc[:, :6]
        data_ratio_ab["Segment"] = "EO"

        for i, stress in enumerate(["AC1", "AC2"]):
            temp = data_ratio.iloc[:, 6 + 4 * i:10 + 4 * i].copy()
            temp["Segment"] = stress
            temp = pd.concat([head, temp], axis=1)
            temp = temp.drop([0])
            data_ratio_ab = pd.concat([data_ratio_ab, temp], ignore_index=True)

        data_ratio_ab2 = data_ratio.iloc[:, 6 + 4 * (i+1):10 + 4 * (i+1)].copy()
        data_ratio_ab2["Segment"] = "EO"
        data_ratio_ab2 = pd.concat([head, data_ratio_ab2], axis=1)
        data_ratio_ab2 = data_ratio_ab2.drop([0])
        i+=1

        for j, stress in enumerate(["AC1", "AC2"]):
            temp = data_ratio.iloc[:, 6 + 4 * (i+1):10 + 4 * (i+1)].copy()
            temp["Segment"] = stress
            temp = pd.concat([head, temp], axis=1)
            temp = temp.drop([0])
            data_ratio_ab2 = pd.concat([data_ratio_ab2, temp], ignore_index=True)


        data_ratio = pd.merge(data_ratio_ab2, data_ratio_ab, on=["Segment", "Subject (No.)"])
        data_ratio = data_ratio.rename(columns={"Gender_x": "Gender"})
        data_ratio = data_ratio.drop(columns=["Gender_y"])


        data = pd.merge(data, data_ratio, on=["Segment", "Subject (No.)"])
        try:
            data = data.rename(columns={"Gender_x": "Gender"})
            data = data.drop(columns=["Gender_y"])
        except:
            pass
    data['Gender'] = data['Gender'].replace({'Male': 0, 'Female': 1})
    data['Segment'] = data['Segment'].replace({'EO': 0, 'AC1': 1, 'AC2': 2})

    print(data.shape)

    # Shuffle the data
    data = data.sample(frac=1)
    print(data)
    # Split the data
    split_index = int(len(data) * 0.7)
    train_data = data[:split_index]
    test_data = data[split_index:]

    # Prepare the input features and target labels
    x_train = train_data.drop(columns=['Segment', 'Subject (No.)']).astype('float32')
    y_train = train_data['Segment']
    x_test = test_data.drop(columns=['Segment', 'Subject (No.)']).astype('float32')
    y_test = test_data['Segment']

    # Convert target labels to one-hot encoding
    num_classes = len(y_train.unique())
    y_train = to_categorical(y_train, num_classes)
    y_test = to_categorical(y_test, num_classes)

    """"""
    #YOU CAN CHANGE THE WHOLE AUGUMENTATION PROCESS

    # Normalize the input features
    x_train = (x_train - x_train.mean()) / x_train.std()
    x_test = (x_test - x_test.mean()) / x_test.std()

    # Function to add Gaussian noise
    def add_gaussian_noise(X, mean=0, std=0.01):
        noise = np.random.normal(mean, std, X.shape)
        X_noisy = X + noise
        return X_noisy

    # Apply Gaussian noise to the training data
    x_train_noisy = add_gaussian_noise(x_train)

    # Combine original and noisy data
    x_train_combined = np.vstack((x_train, x_train_noisy))
    y_train_combined = np.vstack((y_train, y_train))

    # Convert combined data back to non-categorical for SMOTE
    y_train_combined_non_categorical = np.argmax(y_train_combined, axis=1)

    # Apply SMOTE
    smote = SMOTE()
    x_train_final, y_train_final = smote.fit_resample(x_train_combined, y_train_combined_non_categorical)

    print(x_train_final.shape, y_train_final.shape)

    ''''''

    # Convert y_train_final back to categorical
    y_train_final = to_categorical(y_train_final, num_classes)

    # YOU CAN CHANGE THE NUMBER OF NODES IN EACH LAYER, THE NUMBER OF LAYERS, THE ACTIVATION FUNCTIONS, THE REGULARIZATION,
    # THE DROPOUT, THE BATCH NORMALIZATION, THE LEARNING RATE, THE OPTIMIZER, THE LOSS FUNCTION,  THE EPOCHS,
    # THE BATCH SIZE, THE VALIDATION SPLIT, THE LEARNING RATE SCHEDULER, IDK WHAT ELSE THAT'S ALL I KNOW
    # Define the model
    model = Sequential([
        Dense(128, input_shape=(x_train_final.shape[1],), activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.1),
        Dense(64, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.1),
        Dense(32, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dense(16, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.1),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.005),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

    # Learning rate scheduler
    def lr_schedule(epoch):
        lr = 0.0001
        if epoch > 150:
            lr = 0.00005
        elif epoch > 100:
            lr = 0.0001
        elif epoch > 50:
            lr = 0.0002
        print(f"Learning rate: {lr}")
        return lr

    lr_scheduler = keras.callbacks.LearningRateScheduler(lr_schedule)

    # Train the model
    history = model.fit(x_train_final, y_train_final, epochs=200, batch_size=2, validation_split=0.3, callbacks=[lr_scheduler])

    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(x_test, y_test)

    print(f'Test loss: {test_loss:.4f}')
    print(f'Test accuracy: {test_accuracy:.4f}')

    # Plotting training history
    fig, axs = plt.subplots(2)

    # Accuracy subplot
    axs[0].plot(history.history["accuracy"], label="train accuracy")
    axs[0].plot(history.history["val_accuracy"], label="validation accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[0].set_title("Accuracy eval")

    # Loss subplot
    axs[1].plot(history.history["loss"], label="train loss")
    axs[1].plot(history.history["val_loss"], label="validation loss")
    axs[1].set_ylabel("Loss")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")
    axs[1].set_title("Loss eval")

    plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'ECG (EO, AC1, AC2).xlsx'